<a href="https://colab.research.google.com/github/ppaulo17/Mine/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  #!pip install tf-nightly
  pass
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.19.0


In [8]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2025-11-25 14:46:56--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.1’

train-data.tsv.1    100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2025-11-25 14:46:56 (24.6 MB/s) - ‘train-data.tsv.1’ saved [358233/358233]

--2025-11-25 14:46:56--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.2.33, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.1’

valid-data.tsv.1    100%[====

In [9]:
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "msg"])
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "msg"])

train_df['type'] = train_df['type'].map({'ham': 0, 'spam': 1})
test_df['type'] = test_df['type'].map({'ham': 0, 'spam': 1})

train_labels = train_df.pop('type')
test_labels = test_df.pop('type')

encoder = tf.keras.layers.TextVectorization(max_tokens=1000, output_sequence_length=100)
encoder.adapt(train_df['msg'].values)

In [10]:
model=tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True
    ),

    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(
    train_df['msg'].values,
    train_labels.values,
    epochs=10,
    validation_data=(test_df['msg'].values, test_labels),
    verbose=2
)



Epoch 1/10
131/131 - 28s - 211ms/step - accuracy: 0.9304 - loss: 0.2373 - val_accuracy: 0.9777 - val_loss: 0.0727
Epoch 2/10
131/131 - 17s - 129ms/step - accuracy: 0.9832 - loss: 0.0596 - val_accuracy: 0.9856 - val_loss: 0.0482
Epoch 3/10
131/131 - 21s - 157ms/step - accuracy: 0.9899 - loss: 0.0356 - val_accuracy: 0.9806 - val_loss: 0.0538
Epoch 4/10
131/131 - 22s - 166ms/step - accuracy: 0.9943 - loss: 0.0183 - val_accuracy: 0.9871 - val_loss: 0.0582
Epoch 5/10
131/131 - 19s - 147ms/step - accuracy: 0.9969 - loss: 0.0116 - val_accuracy: 0.9828 - val_loss: 0.0729
Epoch 6/10
131/131 - 19s - 147ms/step - accuracy: 0.9945 - loss: 0.0203 - val_accuracy: 0.9813 - val_loss: 0.0772
Epoch 7/10
131/131 - 17s - 132ms/step - accuracy: 0.9983 - loss: 0.0061 - val_accuracy: 0.9806 - val_loss: 0.0710
Epoch 8/10
131/131 - 17s - 130ms/step - accuracy: 0.9983 - loss: 0.0051 - val_accuracy: 0.9756 - val_loss: 0.0826
Epoch 9/10
131/131 - 21s - 157ms/step - accuracy: 0.9974 - loss: 0.0098 - val_accuracy: 

In [11]:
def predict_message(pred_text):
  prediction = model.predict(tf.constant([pred_text]))[0][0]

  label = "ham" if prediction < 0.5 else "spam"


  return [prediction, label]

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 624ms/step
[np.float32(7.1424466e-07), 'ham']


In [12]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 134ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
You passed the challenge. Great job!
